# UnitTests: CayleyTable

In [5]:
import unittest
from unittest import TestCase
from cayley_table import CayleyTable, about_tables
import numpy as np

In [ ]:
class TestCayleyTable(TestCase):

    def setUp(self) -> None:
        # not assoc; is comm; no identity -- the RPS magma table, above
        self.tbl1 = [[0, 1, 0],
                     [1, 1, 2],
                     [0, 2, 2]]
        self.tbl1_copy = [[0, 1, 0],
                          [1, 1, 2],
                          [0, 2, 2]]

        # is assoc; not comm; has identity (0) --- the S3 group table
        self.tbl2 = [[0, 1, 2, 3, 4, 5],
                     [1, 2, 0, 5, 3, 4],
                     [2, 0, 1, 4, 5, 3],
                     [3, 4, 5, 0, 1, 2],
                     [4, 5, 3, 2, 0, 1],
                     [5, 3, 4, 1, 2, 0]]

        # is assoc; is comm; has identity (0) --- the Z4 group table
        self.tbl3 = [[0, 1, 2, 3],
                     [1, 2, 3, 0],
                     [2, 3, 0, 1],
                     [3, 0, 1, 2]]

        # powerset(3) group table
        self.tbl4 = [[0, 1, 2, 3, 4, 5, 6, 7],
                     [1, 0, 4, 5, 2, 3, 7, 6],
                     [2, 4, 0, 6, 1, 7, 3, 5],
                     [3, 5, 6, 0, 7, 1, 2, 4],
                     [4, 2, 1, 7, 0, 6, 5, 3],
                     [5, 3, 7, 1, 6, 0, 4, 2],
                     [6, 7, 3, 2, 5, 4, 0, 1],
                     [7, 6, 5, 4, 3, 2, 1, 0]]

        # Example 1.4.1 in "Groupoids and Smarandache Groupoids", W. B. Vasantha Kandasamy
        self.tbl5 = [[0, 3, 0, 3, 0, 3],
                     [1, 4, 1, 4, 1, 4],
                     [2, 5, 2, 5, 2, 5],
                     [3, 0, 3, 0, 3, 0],
                     [4, 1, 4, 1, 4, 1],
                     [5, 2, 5, 2, 5, 2]]

        # TABLES FOR RINGS & FIELDS:

        # Addition table of the Ring on the powerset of {0, 1}
        self.tbl6a = [[0, 1, 2, 3],
                      [1, 0, 3, 2],
                      [2, 3, 0, 1],
                      [3, 2, 1, 0]]

        # Multiplication table of the Ring on the powerset of {0, 1}
        self.tbl6m = [[0, 0, 0, 0],
                      [0, 1, 0, 1],
                      [0, 0, 2, 2],
                      [0, 1, 2, 3]]

        # Addition table for the Ring based on 2x2 matrices
        self.tbl7a = [[0, 1, 2, 3],
                      [1, 0, 3, 2],
                      [2, 3, 0, 1],
                      [3, 2, 1, 0]]

        # Multiplication table for the Ring based on 2x2 matrices
        self.tbl7m = [[0, 0, 0, 0],
                      [0, 0, 1, 1],
                      [0, 0, 2, 2],
                      [0, 0, 3, 3]]

        # Addition table of autogenerated Ring of integers mod 6
        self.tbl8a = [[0, 1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5, 0],
                      [2, 3, 4, 5, 0, 1],
                      [3, 4, 5, 0, 1, 2],
                      [4, 5, 0, 1, 2, 3],
                      [5, 0, 1, 2, 3, 4]]

        # Multiplication table of autogenerated Ring of integers mod 6
        self.tbl8m = [[0, 0, 0, 0, 0, 0],
                      [0, 1, 2, 3, 4, 5],
                      [0, 2, 4, 0, 2, 4],
                      [0, 3, 0, 3, 0, 3],
                      [0, 4, 2, 0, 4, 2],
                      [0, 5, 4, 3, 2, 1]]

        test_arrays = [self.tbl1, self.tbl2, self.tbl3, self.tbl4, self.tbl5]

        test_arrays_add = [self.tbl6a, self.tbl7a, self.tbl8a]
        test_arrays_mult = [self.tbl6m, self.tbl7m, self.tbl8m]

        self.test_tables = [CayleyTable(tbl) for tbl in test_arrays]
        self.test_tables_add = [CayleyTable(tbl) for tbl in test_arrays_add]
        self.test_tables_mult = [CayleyTable(tbl) for tbl in test_arrays_mult]
        self.all_tables = self.test_tables + self.test_tables_add + self.test_tables_mult

    # ABOUT TEST_TABLES:
    #
    #    Table  Order  Associative?  Commutative?  Left Id?  Right Id?  Identity?  Inverses?
    # -------------------------------------------------------------------------------------
    #      1      3       False         True         None      None       None      False
    #      2      6        True        False            0         0          0       True
    #      3      4        True         True            0         0          0       True
    #      4      8        True         True            0         0          0       True
    #      5      6        True        False         None         0       None      False

    # NOTE: Creating an inverse_lookup_dict requires that the index, "id", of the identity
    # element be provided.

    def test_inverse_lookup_dict_1(self):
        tb1 = self.test_tables[1]
        id1 = 0
        self.assertEqual(tb1.inverse_lookup_dict(id1), {0: 0, 1: 2, 2: 1, 3: 3, 4: 4, 5: 5})

    def test_inverse_lookup_dict_2(self):
        tb2 = self.test_tables[2]
        id2 = 0
        self.assertEqual(tb2.inverse_lookup_dict(id2), {0: 0, 1: 3, 2: 2, 3: 1})

    def test_table_order(self):
        result = [tbl.order for tbl in self.all_tables]
        self.assertEqual(result, [3, 6, 4, 8, 6, 4, 4, 6, 4, 4, 6])
    
    def test_table_associative(self):
        result = [tbl.is_associative() for tbl in self.all_tables]
        self.assertEqual(result, [False, True, True, True, True, True, True, True, True, True, True])

    def test_table_commutative(self):
        result = [tbl.is_commutative() for tbl in self.all_tables]
        self.assertEqual(result, [True, False, True, True, False, True, True, True, True, False, True])

    def test_table_left_id(self):
        result = [tbl.left_identity() for tbl in self.test_tables]
        self.assertEqual(result, [None, 0, 0, 0, None])

    def test_table_right_id(self):
        result = [tbl.right_identity() for tbl in self.test_tables]
        self.assertEqual(result, [None, 0, 0, 0, 0])

    def test_table_identity(self):
        result = [tbl.identity() for tbl in self.test_tables]
        self.assertEqual(result, [None, 0, 0, 0, None])

    def test_table_has_inverses(self):
        result = [tbl.has_inverses() for tbl in self.test_tables]
        self.assertEqual(result, [False, True, True, True, False])

    def test_cayley_table_to_str(self):
        result = str(CayleyTable(self.tbl3))
        self.assertEqual(result[:26], '<CayleyTable, order 4, ID:')

    def test_cayley_table_to_list(self):
        result = CayleyTable(self.tbl3).tolist()
        self.assertEqual(result, [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]])

    def test_cayley_table_to_list_with_names(self):
        result = CayleyTable(self.tbl3).to_list_with_names(["e", "a", "a^2", "a^3"])
        answer = [['e', 'a', 'a^2', 'a^3'],
                  ['a', 'a^2', 'a^3', 'e'],
                  ['a^2', 'a^3', 'e', 'a'],
                  ['a^3', 'e', 'a', 'a^2']]
        self.assertEqual(result, answer)

    def test_equal(self):
        self.assertEqual(CayleyTable(self.tbl1), CayleyTable(self.tbl1_copy))

    def test_mult_distributes_over_add(self):
        result = [m.distributes_over(a) for (m, a) in zip(self.test_tables_mult, self.test_tables_add)]
        self.assertEqual(result, [True, True, True])

    def test_add_distributes_over_mult(self):
        result = [a.distributes_over(m) for (m, a) in zip(self.test_tables_mult, self.test_tables_add)]
        self.assertEqual(result, [False, False, False])

    def test_about_tables(self):
        self.assertEqual(about_tables(self.test_tables), 0)

In [ ]:
unittest.main(argv=[''], verbosity=3, exit=False)